In [1]:
import shutup
shutup.please()

import gym
import src.environments.continuous.stock_trading  

import numpy as np
import pandas as pd 

from tqdm.notebook import trange, tqdm

### Unit tests for the trading environment

In [2]:
def environment():
    env = gym.make('StockTradingEnvironment-v0',
        use_technical_indicators= [
        "macd",
        "boll_ub",
        "boll_lb",
        "rsi_30",
        "cci_30",
        "dx_30",
        "close_30_sma",
        "close_60_sma",
    ])
    
    env.success_threshold =0.25 # 25%
    return env


In [3]:
env = environment()
original_state = env.reset(dataset_id=1);

#### Dataset tests

#### Target tests

#### Target tests

#### Trade tests

#### Reset tests

In [4]:
from collections import deque

In [5]:
env = environment()
original_state = env.reset(dataset_id=1);

In [6]:
orders_history=deque(maxlen=env.lookback)
portfolio_history=deque(maxlen=env.lookback)
market_history=deque(maxlen=env.lookback)

##### 1. State and next state functions

In [7]:
def _state():
    state = np.concatenate((
    orders_history,
    portfolio_history,
    market_history)
    ,axis=1)

    return state

# def _next_state(self):
#     i = env.current_step

#     held = 1
#     if env.stock_sold > 0 or env.stock_bought > 0:
#         held = 0

#     # # Add order tracking
#     self.orders_history.append([held,self.stock_sold,self.stock_bought])

#     # # Add portfoluio state tracking
#     self.portfolio_history.append(self._normalize_portfolio(i))  # % % %

#         # # Market history tracks OHLC
#         self.market_history.append(self.df_norm.iloc[i])

#         self._state()

In [8]:
orders_history.clear()
portfolio_history.clear()
market_history.clear()

#print('Quer-se current_steps de 0 a lookback -1')
current_step = env.lookback
current_steps = []
for i in reversed(range(env.lookback)):
    
    current_step = env.lookback -i -1
    # Orders history tracks recent trader activity - held bought sold
    orders_history.append([0,0,0]) # Held, Sold, Bought
    # Portfolio
    portfolio_history.append([1,1,0,1])  # portfolio_value_% =>  cash_held_% => stocks_held_% stock_price_avg_comp_%

    # Market history 
    market_history.append(env.df_norm.iloc[current_step]) #env.df_norm.iloc[current_step])
    
    current_steps.append(current_step)
    
state = _state()

assert current_steps[0] == 0 , "Wrong first df iloc"
assert current_steps[-1] == env.lookback -1 , "Wrong last df iloc"
assert env.lookback == len(state), "Wrong state length"
assert env.current_step == env.lookback , 'Wrong current step'
assert state.shape==env.observation_space.shape , 'Wrong state vs observation shapes'
assert np.all(state == original_state) , 'Test state doesn\'t match original state'
assert np.all(state[-1,7:] == env.df_norm.iloc[env.current_step-1].values) , 'Normalized technical indicators dont match'
#print(env.current_step, env.lookback, state.shape,env.observation_space.shape)

In [9]:
### NEXT STATE
state = env.step(1)[0]

assert np.all(state[-1,7:] == env.df_norm.iloc[env.current_step-1].values) , 'Normalized technical indicators dont match'

##### Actions

In [10]:
stock_held = []
stock_prices = []

global cash_in_hand
cash_in_hand = env.initial_investment
cash_in_hand

408.4999847412109

In [11]:
assert cash_in_hand == env.cash_in_hand , 'Cash in hand doesnt match'

In [12]:
assert env.extract_action(0) == (0,0) , "Sell action failed"
assert env.extract_action(1) == (1,0), "Hold action failed"
assert env.extract_action(2) == (2,1), "Failed to buy one stock"

In [13]:
assert env.get_current_price() == env.df.iloc[env.current_step -1].close, 'Current price doesnt match'
assert env.get_current_buying_price() == env.df.iloc[env.current_step -1].close * (1+env.fees.BUY) , 'Current buying price doesnt match'
assert env.get_current_selling_price() == env.df.iloc[env.current_step -1].close * (1-env.fees.SELL) , 'Current selling price doesnt match'

In [14]:
# Buy one stock
current_buying_price = env.get_current_buying_price()
def _buy():
    current_price = env.get_current_buying_price()
    stock_held.append(1)
    stock_prices.append(current_price)
    global cash_in_hand
    cash_in_hand -= current_price
    
_buy()    

env.step(2);

assert env.cash_in_hand == cash_in_hand , 'Cash in hand doenst match'
assert env.stock_held == 1, 'Failed to buy stock'
assert env.initial_investment -current_buying_price == cash_in_hand , 'Cash in hand update after buy doesnt match'

In [15]:
# Buy all stocks possible
for i in range(env.maximum_stocks_held-2):
    _buy()
    env.step(2);

# Try to buy past that 
for i in range(5):
    env.step(2);
    
assert env.stock_held == env.maximum_stocks_held -1, 'Failed to buy stock'
assert env.cash_in_hand > 0, 'Cannot be in debt'
assert np.all(stock_prices == env.stock_prices) , 'Stock purchase prices must match'

In [16]:
## means 

In [17]:
assert env.extract_action(0) == (0,env.maximum_stocks_held -1) , "Sell action failed"

transaction_profits = []
def _sell():
    current_price = env.get_current_selling_price()
    for i in range(len(stock_held)):
        sold_price = stock_held[i] * current_price
        bought_price = stock_held[i] * stock_prices[i]
        global cash_in_hand
        cash_in_hand += sold_price
        transaction_profits.append((sold_price-bought_price)/bought_price)
    
sold_price = env.get_current_selling_price()
_sell()
# Sell all stocks
env.step(0)

assert env.stock_held == 0 , 'Sold all stocks'
assert env.cash_in_hand, cash_in_hand == 'Cash in hand doesn\'t match'

In [18]:
transaction_profits

[0.12568711905252092,
 0.11661879394648356,
 0.07338419546303694,
 0.06273794293666936,
 0.043498564531023944,
 0.009597919558718112,
 0.01287686057461776,
 0.016295277552374546,
 0.020929106055074257]

In [19]:
stock_prices

[38.8262859725952,
 39.14160341644287,
 40.718179180145256,
 41.126084083557124,
 41.88434127807617,
 43.2907488746643,
 43.150605667114256,
 43.00546402740478,
 42.81026933288574]

In [20]:
env.trading_history

deque([{'date': Timestamp('2018-02-09 00:00:00'),
        'open': 39.26750183105469,
        'close': 39.102500915527344,
        'high': 39.47249984741211,
        'low': 37.560001373291016,
        'amount_traded': 1,
        'action': 'buy',
        'current_price': 38.8262859725952,
        'step': 1,
        'reward': 0.12568711905252092,
        'portfolio_value': 408.4999847412109},
       {'date': Timestamp('2018-02-12 00:00:00'),
        'open': 39.625,
        'close': 40.6775016784668,
        'high': 40.97249984741211,
        'low': 39.37749862670898,
        'amount_traded': 1,
        'action': 'buy',
        'current_price': 39.14160341644287,
        'step': 2,
        'reward': 0.11661879394648356,
        'portfolio_value': 408.42240974426267},
       {'date': Timestamp('2018-02-13 00:00:00'),
        'open': 40.48749923706055,
        'close': 41.084999084472656,
        'high': 41.1875,
        'low': 40.41249847412109,
        'amount_traded': 1,
        'action':

In [21]:
for i in range(len(env.trading_history)):
    th = env.trading_history[i]
    if th['action']=="buy":
        assert th['current_price'] == stock_prices[i], 'Buying prices dont match'
        
    if th['action']=="sell":
        assert th['current_price'] == sold_price, 'Selling prices dont match'
    
    if th['action']=="sell":
        assert th['current_price'] == sold_price, 'Selling prices dont match'

#### Step tests